In [57]:
# -*- coding: utf-8 -*-
# @Time    : 2022/3/30 20:43
# @Author  : huangkai
# @File    : search_content.py


import sys

sys.path.append('../recall/nlp4es8/ir/')
sys.path.append('../utils')

from es_config import Config


# from utils.logger_config import base_logger
# from utils.args import FLAGS
# from ir.index_hnsw import Index


class Search(object):
    def __init__(self, config, index_name):
        # base_logger.info("Searching ...")
        self.config = config
        self.es = self.config.es

        self.index_name = index_name

    def searchAnswer(self, question, ):
        body = {
            "query": {
                "multi_match": {
                    "query": question,
                    "fields": ["idx_document"],  # 在question字段中匹配查询
                    "type": "most_fields",
                }
            }
        }

        # es相关配置

        res = self.es.search(index=self.index_name, body=body, request_timeout=30, size=self.config.top_n)

        topn = res['hits']['hits']

        result = []
        for data in topn:
            result.append(
                (
                    data['_source']['index'],
                    data['_source']['idx_document'],
                    data['_source']['ori_document']
                )
            )
        return result


In [58]:
import pandas as pd
import json

index_file_content_config = Config()
index_file_content_config.index_name = "kbqa"
index_file_content_config.top_n=20
search_name = Search(index_file_content_config, "kbqa")

In [32]:
import json
dev_data = [json.loads(line.strip()) for line in open("../data/dev_sft.json", encoding="utf8")]
train_data = [json.loads(line.strip()) for line in open("../data/train_sft.json", encoding="utf8")]

In [45]:
import json
train = json.loads( open("../data/train.json", encoding="utf8").read())

In [48]:
train_dic={}
for unit in train:
    train_dic[unit["question"]]=unit["attribute"]

In [47]:
data=train_data+dev_data

In [49]:
from tqdm import tqdm
final_result={}
for unit in tqdm(data):
    question=unit["question"]
    if question in final_result:
        if unit["task_type"]=="tuple_extract":
            final_result[question]["tuple_answer"]=unit["answer"]
        else:
            final_result[question]["table_answer"]=train_dic[question]
            final_result[question]["es_top20"]=unit["instruction"]
    else:
        final_result[question]={}
        if unit["task_type"]=="tuple_extract":
            final_result[question]["tuple_answer"]=unit["answer"]
        else:
            final_result[question]["table_answer"]=train_dic[question]
            final_result[question]["es_top20"]=unit["instruction"]

100%|██████████████████████████████████████████████████████████████████████████| 2349/2349 [00:00<00:00, 784865.78it/s]


In [50]:
final_result["在我的心中，阿杜是一位总能给人带来惊喜的音乐熊。他的嗓音独栋特，曲风新颖，作品里满了他的个人魅力。我记得他曾在新家坡出道，啥知道他唱的是流行歌曲，还讲得他喜爱的书脚是《龙虎》们。但我不知道他最近的搭讪，最开心的原以及他最喜欢的艺人，希望你可以告街我。"]

{'tuple_answer': '阿杜最近的梦想;阿杜最高兴的事;阿杜最喜欢的艺人',
 'table_answer': ['阿杜 ||| 最近的梦想 ||| 过年能去国外旅游~',
  '阿杜 ||| 最高兴的事 ||| 中马票',
  '阿杜 ||| 最喜欢的艺人 ||| 周慧敏'],
 'es_top20': ['原来我最爱的人是你不是他 ||| 别名 ||| 原来我最爱的人是你不是他',
  '原来我最爱的人是你不是他 ||| 中文名称 ||| 原来我最爱的人是你不是他',
  '原来我最爱的人是你不是他 ||| 音乐风格 ||| 流行',
  '他是我最讨厌的人 ||| 别名 ||| 他是我最讨厌的人',
  '我的他不是他 ||| 别名 ||| 我的他不是他',
  '他是我最讨厌的人 ||| 中文名 ||| 他是我最讨厌的人',
  '我的他不是他 ||| 中文名 ||| 我的他不是他',
  '原来你喜欢的是他 ||| 别名 ||| 原来你喜欢的是他',
  '原来我最爱的人是你不是他 ||| 歌曲原唱 ||| 林健辉',
  '我的他是超人 ||| 书名 ||| 我的他是超人',
  '我爱的人，他不是人 ||| 别名 ||| 我爱的人，他不是人',
  '我爱的人，他不是人 ||| 中文名 ||| 我爱的人,他不是人',
  '原来我最爱的人是你不是他 ||| 歌曲语言 ||| 国语',
  '他说我是他的 ||| 别名 ||| 他说我是他的',
  '你是我的他 ||| 别名 ||| 你是我的他',
  '原来你喜欢的是他 ||| 中文名称 ||| 原来你喜欢的是他',
  '他爱的人不该是我 ||| 别名 ||| 他爱的人不该是我',
  '我的他是超人 ||| 别名 ||| 我的他是超人',
  '他说我是他的 ||| 中文名 ||| 他说我是他的',
  '他，是我的 ||| 别名 ||| 他，是我的']}

In [51]:
dev_result={}
for unit in tqdm(dev_data):
    if unit["task_type"]=="tuple_extract":
        dev_result[unit["question"]]=final_result[unit["question"]]

100%|████████████████████████████████████████████████████████████████████████████| 470/470 [00:00<00:00, 471382.80it/s]


In [55]:
dev_result["什么事情让莫文蔚感到感动？"]

{'table_answer': ['莫文蔚 ||| 最感动的事 ||| 歌迷做的网页'],
 'es_top20': ['《动什么，别动感情》 ||| 别名 ||| 《动什么，别动感情》',
  '《动什么别动感情》 ||| 别名 ||| 《动什么别动感情》',
  '还有什么样的感情能让你心动 ||| 中文名 ||| 还有什么样的感情能让你心动',
  '还有什么样的感情能让你心动 ||| 别名 ||| 还有什么样的感情能让你心动',
  '什么是感动 ||| 中文名 ||| 什么是感动',
  '动什么，别动感情(赵赵著小说) ||| 别名 ||| 动什么，别动感情',
  '动什么，别动感情(赵赵著小说) ||| 书名 ||| 动什么，别动感情',
  '什么是感动 ||| 别名 ||| 什么是感动',
  '什么都不动也得动感情 ||| 别名 ||| 什么都不动也得动感情',
  '什么都不动也得动感情 ||| 书名 ||| 什么都不动也得动感情',
  '是什么样的感情 ||| 中文名 ||| 是什么样的感情',
  '拿什么感动你 ||| 书名 ||| 拿什么感动你',
  '拿什么感动你 ||| 别名 ||| 拿什么感动你',
  '为什么感到不幸福 ||| 书名 ||| 为什么感到不幸福',
  '为什么感到不幸福 ||| 别名 ||| 为什么感到不幸福',
  '动什么，别动感情(2004年唐大年执导电视剧) ||| 中文名 ||| 动什么，别动感情',
  '是什么样的感情 ||| 别名 ||| 是什么样的感情',
  '我用什么来感动 ||| 中文名 ||| 我用什么来感动',
  '是什么左右了感情 ||| 中文名 ||| 是什么左右了感情',
  '动什么，别动感情(2004年唐大年执导电视剧) ||| 别名 ||| 动什么，别动感情'],
 'tuple_answer': '莫文蔚最感动的事'}

In [56]:
submission = open("es召回测试.json", "w", encoding="utf8")

submission.write(json.dumps(dev_result, ensure_ascii=False))

137936

In [25]:
from tqdm import tqdm
count,sums=0,0
for unit in tqdm(dev_data):
    if unit["task_type"]=="tuple_extract":
    for attr in unit["attribute"]:
        sums+=1
        if attr in [k[2] for k in search_name.searchAnswer(unit["question"])]:
            count+=1

  9%|███████▍                                                                        | 93/1000 [02:12<21:30,  1.42s/it]


KeyboardInterrupt: 

In [22]:
unit["attribute"]

['于丹现象 ||| 人物职业 ||| 北京师范大学教授',
 '于丹现象 ||| 产生地点 ||| 央视百家讲坛',
 '于丹现象 ||| 产生原因 ||| 七天解读《论语》心得']

In [14]:
count/sums

0.1506504406210659

In [20]:
result = search_name.searchAnswer(unit["question"])

In [21]:
result

[('26449151', '我是一个不懂倾诉的对象别名', '我是一个不懂倾诉的对象 ||| 别名 ||| 我是一个不懂倾诉的对象'),
 ('614529', '于丹现象产生原因', '于丹现象 ||| 产生原因 ||| 七天解读《论语》心得'),
 ('27241571', '这些机会不是属于我的别名', '这些机会不是属于我的 ||| 别名 ||| 这些机会不是属于我的'),
 ('26449154', '我是一个不懂倾诉的对象歌曲原唱', '我是一个不懂倾诉的对象 ||| 歌曲原唱 ||| 郑嘉颖'),
 ('27241575', '这些机会不是属于我的填词', '这些机会不是属于我的 ||| 填词 ||| 梁栢坚'),
 ('27241576', '这些机会不是属于我的谱曲', '这些机会不是属于我的 ||| 谱曲 ||| Michael'),
 ('614528', '于丹现象产生地点', '于丹现象 ||| 产生地点 ||| 央视百家讲坛'),
 ('12922723', '这些年我深爱的女孩别名', '这些年我深爱的女孩 ||| 别名 ||| 这些年我深爱的女孩'),
 ('27241574', '这些机会不是属于我的歌曲原唱', '这些机会不是属于我的 ||| 歌曲原唱 ||| Kolor'),
 ('21350956', '这是我的人生，不是别人的ISBN', '这是我的人生，不是别人的 ||| ISBN ||| 9787508629766'),
 ('14780140', '这个女人是我的别名', '这个女人是我的 ||| 别名 ||| 这个女人是我的'),
 ('21198484', '这些都是我的爱制作人', '这些都是我的爱 ||| 制作人 ||| 莫小滢'),
 ('12922725', '这些年我深爱的女孩作者', '这些年我深爱的女孩 ||| 作者 ||| 谢言社'),
 ('26449152', '我是一个不懂倾诉的对象中文名称', '我是一个不懂倾诉的对象 ||| 中文名称 ||| 我是一个不懂倾诉的对象'),
 ('26449153', '我是一个不懂倾诉的对象所属专辑', '我是一个不懂倾诉的对象 ||| 所属专辑 ||| 郑嘉颖 新歌+精选'),
 ('26449155', '我是一个不懂倾诉的对象歌曲语言', '我是一

In [8]:
err=0
for unit in train_data:
    for attr in unit["attribute"]:
        
        if attr not in [k[1] for k in unit["top20"]]:
            err+=1
            break

In [9]:
err,len(train_data)

(683, 1000)

In [10]:
1-err/len(train_data)

0.31699999999999995

In [11]:
unit

{'id': 'TR002899',
 'question': '近些年，我非常关注于丹现象。我对这种深入传统，大众化宣传的方式深感兴趣。但我对于丹详细的个人信息了解不多，特别是他的个人职业，现象的产生地点，以及产生原因等等，您能告诉我一下这些信息ma？',
 'attribute': ['于丹现象 ||| 人物职业 ||| 北京师范大学教授',
  '于丹现象 ||| 产生地点 ||| 央视百家讲坛',
  '于丹现象 ||| 产生原因 ||| 七天解读《论语》心得'],
 'top20': [['26449151', '我是一个不懂倾诉的对象 ||| 别名 ||| 我是一个不懂倾诉的对象'],
  ['27241571', '这些机会不是属于我的 ||| 别名 ||| 这些机会不是属于我的'],
  ['26449152', '我是一个不懂倾诉的对象 ||| 中文名称 ||| 我是一个不懂倾诉的对象'],
  ['12922723', '这些年我深爱的女孩 ||| 别名 ||| 这些年我深爱的女孩'],
  ['27241572', '这些机会不是属于我的 ||| 单曲名称 ||| 这些机会不是属于我的'],
  ['12922724', '这些年我深爱的女孩 ||| 中文名 ||| 这些年我深爱的女孩'],
  ['14780140', '这个女人是我的 ||| 别名 ||| 这个女人是我的'],
  ['21198481', '这些都是我的爱 ||| 别名 ||| 这些都是我的爱'],
  ['9416371', '关于我的一些事 ||| 别名 ||| 关于我的一些事'],
  ['12928294', '对于他，这是个不能说的秘密 ||| 别名 ||| 对于他，这是个不能说的秘密'],
  ['30246526', '僵尸员工 ||| 产生原因 ||| 产生这种现象的原因是多'],
  ['19435548', '我就是这样的一个人 ||| 别名 ||| 我就是这样的一个人'],
  ['21350954', '这是我的人生，不是别人的 ||| 别名 ||| 这是我的人生，不是别人的'],
  ['21350955', '这是我的人生，不是别人的 ||| 书名 ||| 这是我的人生，不是别人的'],
  ['14780141', '这个女人是我的 ||| 

In [87]:
# v2 test
import json
from tqdm import tqdm
train = json.loads( open("../result/es召回测试_v2.json", encoding="utf8").read())

In [88]:
for unit in tqdm(train):
    tuple_=train[unit]["tuple_predict"]
    train[unit]["es_top20"]=[k[2] for k in search_name.searchAnswer(tuple_)]

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:09<00:00, 17.16it/s]


In [89]:
sums,count,final=0,0,0
for unit in train:
    for tb in train[unit]["table_answer"]:
        sums+=1
        if tb in train[unit]["es_top20"]:
            count+=1
        else:
            print(tb,"=====",train[unit]["es_top20"],"\n")
            break
    if set(train[unit]["table_answer"]).issubset(set(train[unit]["es_top20"])):
        final+=1

葛珊珊 ||| 赵本山徒弟 ||| 超本山收下的第五批徒弟 ===== ['葛珊珊 ||| 生肖 ||| 猪', '葛珊珊 ||| 丈夫 ||| 历小峰', '葛珊珊 ||| 籍贯 ||| 吉林省德惠市', '葛珊珊 ||| 师傅 ||| 赵本山', '葛珊珊 ||| 国籍 ||| 中国', '葛珊珊 ||| 星座 ||| 狮子座', '葛珊珊 ||| 血型 ||| O', '葛珊珊 ||| 民族 ||| 汉族', '葛珊珊 ||| 职业 ||| 二人转演员 电视演员', '葛珊珊 ||| 别名 ||| 葛珊珊', '葛珊珊 ||| 性别 ||| 女', '葛珊珊 ||| 出生地 ||| 吉林省德惠市', '葛珊珊 ||| 经纪公司 ||| 本山传媒', '葛珊珊 ||| 中文名 ||| 葛珊珊', '葛珊珊 ||| 英文名 ||| GeShanShan', '葛珊珊 ||| 出生日期 ||| 1983年8月17日', '葛珊珊 ||| 家庭成员 ||| 历小峰（丈夫）', '葛珊珊 ||| 签约公司 ||| 本山传媒', '葛珊珊 ||| 主要成就 ||| 赵本山老师的第五批徒弟', '葛珊珊 ||| 代表作品 ||| 乡村爱情'] 

三上枝织 ||| 中文名 ||| 三上枝织 ===== ['三上枝织 ||| 所属团体 ||| 七森中☆娱乐部', '三上枝织 ||| 毕业院校 ||| 东京广播学院', '三上枝织 ||| 经纪公司 ||| 青二Production', '三上枝织 ||| 代表作品 ||| 《摇曳百合》赤座灯里', '三上枝织 ||| 职业 ||| 声优', '三上枝织 ||| 职 业 ||| 声优', '三上枝织 ||| 出生 ||| 1989年1月6日 （25岁） \xa0日本青森县', '三上枝织 ||| 出道作品 ||| 女学生（守护甜心）', '三上枝织 ||| 出道时间 ||| 2008年', '三上枝织 ||| 民族 ||| 大和', '三上枝织 ||| 国籍 ||| 日本', '三上枝织 ||| 星座 ||| 摩羯座', '三上枝织 ||| 身高 ||| 160cm', '三上枝织 ||| 奖项 ||| 第6届声优奖“新人女优赏”', '三上枝织 ||| 别名 ||| 三上枝织', '三上枝织 ||| 血型 ||| O型', '

In [90]:
count,sums,final

(314, 350, 128)

In [91]:
count/sums

0.8971428571428571

In [72]:
train[unit]["table_answer"]

['阮福澜 ||| 墓葬 ||| 长延陵']

In [73]:
train[unit]["es_top20"]

['阮福澜 ||| 墓葬 ||| 长延陵',
 '阮福澜 ||| 子 ||| 阮福濒',
 '阮福澜 ||| 尊号 ||| 上主',
 '阮福澜 ||| 庙号 ||| 神祖→神宗',
 '阮福澜 ||| 谥号 ||| 威断神武仁昭王→ 承基缵统刚明雄毅威断英武孝昭王→ 承基缵统刚明雄毅威断英武孝昭皇帝',
 '阮福澜 ||| 别名 ||| 阮福澜',
 '阮福澜 ||| 时代 ||| 南北朝时代',
 '阮福澜 ||| 正室 ||| 段氏',
 '阮福澜 ||| 封号 ||| 仁郡公',
 '阮福澜 ||| 出生 ||| 1601年',
 '阮福澜 ||| 逝世 ||| 1648年',
 '阮福澜 ||| 嫡母 ||| 阮氏',
 '阮福澜 ||| 亲属 ||| 阮福源',
 '阮福濒 ||| 前任：阮福澜 继任：阮福溱 ||| 前任：阮福澜 继任：阮福溱',
 '阮福澜 ||| 前任：阮福源 继任：阮福濒 ||| 前任：阮福源 继任：阮福濒',
 '阮福源 ||| 前任：阮潢 继任：阮福澜 ||| 前任：阮潢 继任：阮福澜',
 '阮福映 ||| 陵墓 ||| 天授陵',
 '阮福晈 ||| 陵墓 ||| 孝陵',
 '阮福溱 ||| 墓葬 ||| 长茂陵',
 '阮福源 ||| 墓葬 ||| 长衍陵']

In [75]:
set(train[unit]["table_answer"]).issubset(set(train[unit]["es_top20"]))

True